In [1]:
%run Latex_macros.ipynb

<IPython.core.display.Latex object>

# RNN vanishing/exploding gradient problem

## Training Deep Networks is hard: Review
As we learned in the module on Vanishing and Exploding Gradients
- Training a very deep (many layer) network is difficult
- Because as the gradient flows backwards (from Loss layer to Input layer)
- The Loss Gradients successively either diminish or expand

Let's quickly review the issue of vanishing and exploding gradients.

Here is the picture of gradient flow during Back propagation:

<div>
<center>Backward pass: Loss to Weights</center>
<br>
<img src="images/NN_Layers_plus_Loss_backward.png">
</div>

The Loss Gradient of layer $\ll$

$$\loss'_\llp = \frac{\partial \loss}{\partial \y_\llp}$$ 

flows backwards from Loss Layer $(L+1)$ inductively as:

$$
\begin{array}[lll] \\
\loss'_{(\ll-1)} & = & \frac{\partial \loss}{\partial \y_{(\ll-1)}} \\
         & = & \frac{\partial \loss}{\partial \y_\llp} \frac{\partial \y_\llp}{\partial \y_{(\ll-1)}} \\
         & = & \loss'_\llp \frac{\partial \y_\llp}{\partial \y_{(\ll-1)}}
\end{array}
$$


Moreover, from the Loss Gradient and a local gradient $\frac{\partial \y_\llp}{\partial \W_\llp}$ at layer $\ll$
- We can compute the derivative of the loss with respect to the layer's weights
- Which is used in the update equation for Gradient Descent
- To modify the estimate of the layer's weights
- In the direction of decreasing Loss

$$
\begin{array}[lll] \\
\frac{\partial \loss}{\partial \W_\llp} & = & \frac{\partial \loss}{\partial \y_\llp} \frac{\partial \y_\llp}{\partial \W_\llp} & = & \loss'_\llp \frac{\partial \y_\llp}{\partial \W_\llp}
\end{array}
$$

<div>
<center>Forward and Backward pass: Detail</center>
<br>
<img src="images/Backward_pass_detail.png">
</div>

The issue arises in the second term $\frac{\partial \y_\llp}{\partial \y_{(\ll-1)}}$ of the inductive update of the Loss Gradient

$$
\begin{array}[lll] \\
\loss'_{(\ll-1)} 
         & = & \loss'_\llp \frac{\partial \y_\llp}{\partial \y_{(\ll-1)}}
\end{array}
$$

Since
$$
\y_\llp = a_\llp \left( f_\llp( \y_{(\ll-1)}, \W_{\llp}) \right) 
$$

The derivative
$$
\begin{array}[lllll] \\
\frac{\partial \y_\llp}{\partial \y_{(\ll-1)}}
                                      & = a'_\llp f'_\llp
\end{array}
$$

where
$$
\begin{array}[lll] \\
a'_\llp & = & \frac{\partial a_\llp ( f_\llp(\y_{(\ll-1)}, \W_\llp) )}{\partial f_\llp(\y_{(\ll-1)}, \W_\llp)}  & \text{derivative of } a_\llp(\ldots) \text{ wrt } f_\llp(\ldots)\\
f'_\llp & = & \frac{\partial f_\llp(\y_{(\ll-1)}, W_\llp)}{\partial \y_{(\ll-1)}} & \text{derivative of } f_\llp(\ldots) \text{ wrt } \y_{(\ll-1)}\\
\end{array}
$$


Substituting the value of the loss gradient into the backward update rule:

$$
\begin{array}[llll]\\
\loss'_{(\ll-1)} & = &  \loss'_\llp \frac{\partial \y_\llp}{\partial \y_{(\ll-1)}} \\
         & = &  \loss'_\llp a'_\llp f'_\llp
\end{array}
$$

We see that the backwards step from Loss Gradient of layer $\ll$ to Loss Gradient of layer $(\ll-1)$ introduces
$a'_\llp$ as a multiplicative term.

But as we continue backwards (expanding $\loss'_\llp$ on the right hand side)
we accumulate this multiplicative term

Starting from layer $(L+1)$ and proceeding backwards to layer $\ll$, the Loss Gradient term looks like
$$\loss'_\llp  =   \loss'_{(L+1)} \prod_{l'=\ll+1}^L  a'_{(l')} f'_{(l')}$$

Specifically: it is the $a'_\llp$ term that is problematic
- If the activation functions $a_\llp$ is such that $a'_\llp < 1$:
    - The backwards pass attenuates the Loss Gradient
    - Eventually making it go to $0$ (disappear)
- If the activation function $a_\llp$ is such that $a'_\llp > 1$:
 - The backwards pass amplifies the Loss Gradient
    - Eventually making it go to $\infty$ (explode)

Recall that 
- For $a_\llp = \sigma$ (the sigmoid function)
- $\max{z} a'_\llp(z) = 0.25$  

so using the sigmoid as the default activation
- Made training of deep networks very difficult
- Which stifled progress in Deep Learning

# An unrolled RNN is a Deep Network

If we unroll an RNN that has an input sequence of length $T$
$$
\x_{(1)}, \ldots, \x_{(T)}
$$

we wind up with a network of $T$ layers (plus the Loss layer)

<table>
    <tr>
        <th><center>RNN Loss Gradient Flow</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_layer_loss_gradient.png"></td>
    </tr>
</table>

As the input sequence length $T$ gets large
- It should be no surprise that training an RNN
- Is exposed to the problem of vanishing and exploding gradients
- Because of the derivative of the activation function (written as $\phi$ rather than $a_\llp$ in the RNN literature)

But it turns out that there is a *second* source of vanishing/exploding gradients for RNN's:
- The weight matrix $\W$ is shared at every step of the unrolled network

Let's see how this can lead to vanishing/exploding gradients.

# Vanishing/Exploding gradients

Let's recall the RNN update equations:
$$
\begin{array}[lll]\\
\h_\tp & = & \phi(\W_{xh}\x_\tp  + \W_{hh}\h_{(t-1)}  + \b_h) \\
\y_\tp & = &  \W_{hy} \h_\tp  + \b_y \\
\end{array}
$$

For simplicity of presentation: we will assume activation function $\phi$ is the identity function in this section.


Returning to the equation that derives the derivative of the Loss with respect to weights $\W$:
$$
\begin{array}[lll] \\
\frac{\partial \loss}{\partial \W_\llp} & = & \frac{\partial \loss}{\partial \y_\llp} \frac{\partial \y_\llp}{\partial \W_\llp} & = & \loss'_\llp \frac{\partial \y_\llp}{\partial \W_\llp}
\end{array}
$$

Let's focus on the term
$$\frac{\partial \y^\ip_\tp}{\partial \W}$$

(replacing $\ll$ as the index of the layer with $t$, the time step)

We will focus on the part of $\W$ that is $\W_{hh}$

$$
\frac{\partial \y_\tp}{\partial \W_{hh}} = \frac{\partial \y_\tp}{\partial \h_\tp} \frac{{\partial \h_\tp}}{\partial \W_{hh}}
$$

This term comes about due to the RNN update equation
$$\y_\tp  =  \W_{hy} \h_\tp  + \b_y$$
- And $\h_\tp$ is a function of $\W_{hh}$

$$
\frac{\partial \y^\ip_\tp}{\partial \W} = 
\frac{\partial \y^\ip_\tp}{\partial \W_{hy}}
+
\frac{\partial \y^\ip_\tp}{\partial \h_{(\tt)} } \frac{\partial \h_{(\tt)}}{\partial \W_{hh}}
$$

Let's expand the term
$$
\frac{\partial \h_\tp}{\partial \W_{hh}}
$$

Since 
$$
\h_\tp =  \W_{xh}\x_\tp  + \W_{hh}\h_{(\tt-1)}  + \b_h
$$

$\h_\tp$ depends on $\h_{(\tt-1)}$, which by recursion depends on $\h_{(\tt-2)}$ which $\ldots$ depends on $\h_{(0)}$.
- and all $\h_\tp$ share the *same* $\W_{hh}$.

This means that $\h_\tp$ depends on $\W_{hh}$ through *each* $\h_{(\tt-k)}$ for $k=1, \ldots, \tt$.
$$
\frac{\partial \h_\tp}{\partial \W_{hh}} = 
\sum_{k=1}^{\tt} { 
 \frac{\partial \h_\tp}{\partial \h_{(\tt -k)}}   \frac{\partial \h_{(\tt -k)}}{\partial \W_{hh}}   
}
$$

The problematic term for us is
$$
\frac{\partial \h_\tp}{\partial \h_{(\tt-k)}}
$$

It can be computed by $k$ applications of the Chain Rule as
$$
\begin{array}\\
\frac{\partial \h_\tp}{\partial \h_{(\tt-k)}} &  = & \frac{\partial \h_\tp}{\partial \h_{(\tt -1)}}   \frac{\partial \h_{(\tt -1)}}{\partial \h_{(\tt -2)}} \ldots \frac{\partial \h_{(\tt -k + 1)}}{\partial \h_{(\tt -k)}}\\
& = & \prod_{u=0}^{k-1} { \frac{\partial \h_{(\tt -u)}}{\partial \h_{(\tt - u -1) }}} \\
\end{array}
$$

Each term
$$
\frac{\partial \h_{(\tt -u)}}{\partial \h_{(\tt - u-1)}}
$$
results in a term $\W_{hh}$ because

$$
\h_\tp =  \W_{xh}\x_\tp  + \W_{hh}\h_{(\tt-1)}  + \b_h
$$

So the **repeated product is equal to the matrix $\W_{hh}$ raised to the power $k$**



For simplicity, suppose $\W_{hh}$ were a scalar (in general: use eigenvalues of matrices and matrix algebra)

Raising $\W_{hh}$ to the power of $k$
- Approaches $0$ as $k$ increases, when $\W_{hh} < 1$
- Approaches $\infty$ as $k$ increases, when $\W_{hh} > 1$

In other words:
- As the distance $k$ between time steps increases
- The Loss Gradient tends to either vanish or explode
- Inhibiting weight updates and learning


If updates *do* occur, they will either be
- Erratic (large loss gradients)
- Slow (small loss gradients)

Remember that this cause of vanishing/exploding gradients *is particular to* recurrent layers
- Because of the sharing of weights between time steps
                             

**Aside**

How is raising a matrix to a power related to eigenvalues ?

Consider matrix $M$.  It's eigen decomposition is
$$
M = \W \Lambda \W^{-1}
$$

where $\Lambda$ is the *diagonal* matrix of eigenvalues.

$$
\begin{array} \\
M^p & = & M M^{p-1} \\
& = &  (\W \Lambda \W^{-1}) &  M^{p-1}  & \text{since } M = (\W \Lambda \W^{-1})\\
& = &  (\W \Lambda \W^{-1}) & M M^{p-2} & \text{since }  M^{p-1} = M M^{p-2} \\
& = &  (\W \Lambda \W^{-1}) & (\W \Lambda \W^{-1}) M^{p-2}  & \text{since } M = (\W \Lambda \W^{-1})\\\\
& = &  (\W \Lambda \W^{-1}  \W \Lambda \W^{-1}) M^{p-2} & \text{associativity of multiplication} \\
& = &  (\W \Lambda^2 \W^{-1}) M^{p-2}  & \text{since } \W \W^{-1} = I, \Lambda \Lambda = \Lambda^2 \\
\vdots
\end{array}
$$

So you can see that raising $M$ to the power $p$ results in diagonal matrix $\Lambda$ being raised to $p$
- \Which is just a diagonal matrix whose elements are the *scalar* diagonal elements of $\Lambda$ raised to $p$

# Controlling exploding gradients by clipping
In theory, we can control the explosion by clipping the gradient $\frac{\partial \loss}{\partial W_i}$.

We are still left with the vanishing gradient problem.

This means that "vanilla" RNN's
have difficulty learning
long-term dependencies (i.e., too many steps backward).


# Conclusion

Recurrent layers are especially exposed to the problem of Vanishing and Exploding gradients
- As potentially very deep networks in the unrolled form
- Due to sharing weights $\W$ across time steps

We will introduce some architectural innovations in Recurrent layers to ameliorate this problem.


In [2]:
print("Done")

Done
